In [977]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd

In [978]:
# Constants

gammac = 1.4
gammah = 1.33

cpa = 1.005
cp = 1.148
R = 287

In [979]:
# Compressor Values

#Ca4 = 226.4117 # m/s
Ca4 = 270
Ca5 = Ca4
N = 208.05 # Rev/s
deltaTocomp = 439.58 # K

In [980]:
To4 = 2275 # K
polyt = 0.92
#flowcoeff Greater than 0.78
#loadcoeff Cannot Exceed 3.3
mfahot = 40 # kg/s
mfthot = mfahot + (mfahot*.02)

In [981]:
# Turbine PARAMETERS

htratioturbine = 0.74

In [982]:
deltaToturb = (deltaTocomp*cpa*0.99)/cp

T4 = To4-((Ca4**2)/(2*cp*1000))

po4 = 15 # bar
p4 = po4*(T4/To4)**(gammah/(gammah-1))

density4 = (p4*100000)/(R*T4)

print(density4)

2.2015640010505257


In [983]:
A4 = mfthot/(density4*Ca4)

M4 = Ca4/(np.sqrt(1.333*R*To4))

print("Turbine Inlet Area = %3.4f m^2" % (A4))

print("Turbine Inlet Mach Number = %3.2f" % (M4))

Turbine Inlet Area = 0.0686 m^2
Turbine Inlet Mach Number = 0.29


In [984]:
r4t = np.sqrt(mfthot/(np.pi*density4*Ca4*(1-(htratioturbine**2))))

r4h = htratioturbine*r4t

r4m = (r4t+r4h)/2

print("r4t = %3.3f m" % (r4t))
print("rm = %3.3f m" % (r4m))
print("r4h = %3.3f m" % (r4h))

r4t = 0.220 m
rm = 0.191 m
r4h = 0.163 m


In [985]:
To5 = To4 - deltaToturb

T5 = To5 - (Ca5**2)/(2*cp*1000)

po5 = po4*((To5/To4)**((gammah*polyt)/(gammah-1)))

p5 = po5*(T5/To5)**(gammah/(gammah-1))

density5 = (p5*100000)/(R*T5)

A5 = mfthot/(density5*Ca5)

print("Turbine Exhaust Area = %3.4f m^2" % (A5))

Turbine Exhaust Area = 0.1137 m^2


In [986]:
# constant mean radius
#bladeheight5 = A5/(2*np.pi*rm)
#
#r5t = rm+(bladeheight5/2)
#
#r5h = rm-(bladeheight5/2)


# constant hub radius
r5h = r4h
r5t = np.sqrt(A5/np.pi + (r5h)**2)
r5m = (r5t+r5h)/2

print("r5t = %3.3f m" % (r5t))
print("r5m = %3.3f m" % (r5m))
print("r5h = %3.3f m" % (r5h))

r5t = 0.250 m
r5m = 0.206 m
r5h = 0.163 m


In [987]:
U5h = r5h*(2*np.pi*N)

print(U5h)

flow5 = Ca5/U5h

print(flow5)

bladeloading = 3.3

print(bladeloading)

deltaTos = (bladeloading*(U5h**2))/(2*cp*1000)

print(deltaTos)

stageest = deltaToturb/deltaTos

print("Estimation of %3.1f Stages" % (stageest))

212.58157265609796
1.2701006800659538
3.3
64.95211350553203
Estimation of 5.9 Stages


In [988]:
U4t = 2*np.pi*r4t*N
U4m = 2*np.pi*r4m*N
U4h = 2*np.pi*r4h*N

In [989]:
stageactual = 5
# Values per stage: alpha2, alpha3, beta2, beta3, r, deltaT, T3, U, Cw2, Cw3, V2, V3, pr, po3, bladeloading, flowcoeff
stagevals = np.zeros((stageactual+1,3,17))
stagevals[0,:,:] = np.array([[0,0,0,0,0,0,To4,U4h,Ca4,0,0,0,0,0,po4,0,0],[0,0,0,0,0,0,To4,U4m,Ca4,0,0,0,0,0,po4,0,0],[0,0,0,0,0,0,To4,U4t,Ca4,0,0,0,0,0,po4,0,0]])

In [990]:
def initialvals4(deltaTocomp,Ca4,To4,po4,htratioturbine):
    deltaToturb = (deltaTocomp*cpa*0.99)/cp

    T4 = To4-((Ca4**2)/(2*cp*1000))

    po4 = 15 # bar
    p4 = po4*(T4/To4)**(gammah/(gammah-1))

    density4 = (p4*100000)/(R*T4)

    A4 = mfthot/(density4*Ca4)

    M4 = Ca4/(np.sqrt(1.333*R*To4))

    r4t = np.sqrt(mfthot/(np.pi*density4*Ca4*(1-(htratioturbine**2))))

    r4h = htratioturbine*r4t

    r4m = (r4t+r4h)/2

    return deltaToturb,r4t,r4h,r4m

In [991]:
def initialvals5(deltaToturb,Ca5,To4,po4,r4h):
    To5 = To4 - deltaToturb

    T5 = To5 - (Ca5**2)/(2*cp*1000)

    po5 = po4*((To5/To4)**((gammah*polyt)/(gammah-1)))

    p5 = po5*(T5/To5)**(gammah/(gammah-1))

    density5 = (p5*100000)/(R*T5)

    A5 = mfthot/(density5*Ca5)

    r5h = r4h

    r5t = np.sqrt(A5/np.pi+(r5h)**2)

    r5m = (r5t+r5h)/2

    return r5t,r5h,r5m

In [992]:
def estimatevals(r5h,N,Ca5,bladeloading,deltaToturb):
    U5h = r5h*(2*np.pi*N)

    flow5 = Ca5/U5h

    deltaTos = (bladeloading*(U5h**2))/(2*cp*1000)

    stageest = deltaToturb/deltaTos

    return stageest

$$
\psi = \frac{2 c_p \Delta T_{0s}}{U^2} \\
\phi = \frac{C_a}{U} \\
tan \beta_2 = \frac{1}{2 \phi} (\frac{\psi}{2} - 2\Lambda) \\
tan \beta_3 = \frac{1}{2 \phi} (\frac{\psi}{2} + 2\Lambda) \\
$$

In [993]:
def returnstage(deltaTo,U,Ca,DOR):
    bladeloading = (2*cp*1000*deltaTo)/(U**2)
    flow = Ca/U
    beta2 = np.degrees(np.arctan((1/(2*flow))*((bladeloading/2)-(2*DOR))))
    beta3 = np.degrees(np.arctan((1/(2*flow))*((bladeloading/2)+(2*DOR))))
    alpha2 = np.degrees(np.arctan(np.tan(np.radians(beta2))+(1/flow)))
    alpha3 = np.degrees(np.arctan((np.tan(np.radians(beta3))-(1/flow))))

    Cw2 = Ca*np.tan(np.radians(alpha2))
    Cw3 = Ca*np.tan(np.radians(alpha3))

    V2 = Ca/(np.cos(np.radians(beta2)))
    V3 = Ca/(np.cos(np.radians(beta3)))

    return beta2,beta3,alpha2,alpha3,Cw2,Cw3,V2,V3,bladeloading,flow

In [994]:
def iterate(stagevals,stageactual,deltaTosact,r4t,r4m,r4h,N,Ca):
    i = 1

    # This runs through the params for each stage and saves them to an array
    while i < stageactual+1:
        if i == 0:
            DOR = 0.5
        if i == 1:
            DOR = 0.55
        if i == 2:
            DOR = 0.6
        if i == 3:
            DOR = 0.7
        else:
            DOR = 0.8

        rtstage = ((r5t-r4t)/(stageactual-1))*i+r4t 
        rmstage = ((r5m-r4m)/(stageactual-1))*i+r4m
        rhstage = ((r5h-r4h)/(stageactual-1))*i+r4h

        Utstage = 2*np.pi*rtstage*N
        Umstage = 2*np.pi*rmstage*N
        Uhstage = 2*np.pi*rhstage*N

        stageTo3 = stagevals[i-1,0,6] + deltaTosact

        # I PROBABLY NEED CHANGED
        pr = (1+((polyt*deltaTosact)/(stagevals[i-1,0,6])))**(gammac/(gammac-1))

        # DO NOT TOUCH THE COMMENTED FUNCTION UNLESS THE FIRST STAGE VALUES ARE FUCKED UP

        """if i == 1:
            deltawhirlt = ((1.005*(10**3)*(stagedeltaTo))/(lambcomp*Utstage))
            Cw2t = deltawhirlt - Cw1t
            stagebeta1tip = np.degrees(np.arctan(Utstage/Castage))
            stagebeta2tip = np.degrees(np.arctan((Utstage-Cw2t)/Castage))
            stagealpha1tip = 0
            stagealpha2tip = np.degrees(np.arctan((Cw2t)/Castage))

            V1t = Castage/(np.cos(np.radians(stagebeta1tip)))
            dehallert = np.cos(np.radians(stagebeta1tip))/np.cos(np.radians(stagebeta2tip))
            V2t = V1t*dehallert

            deltawhirlm = ((1.005*(10**3)*(stagedeltaTo))/(lambcomp*Umcomp))
            Cw2m = deltawhirlm - Cw1m
            stagebeta1mean = np.degrees(np.arctan(Umcomp/Castage))
            stagebeta2mean = np.degrees(np.arctan((Umcomp-Cw2m)/Castage))
            stagealpha1mean = 0
            stagealpha2mean = np.degrees(np.arctan((Cw2m)/Castage))

            V1m = Castage/(np.cos(np.radians(stagebeta1mean)))
            dehallerm = np.cos(np.radians(stagebeta1mean))/np.cos(np.radians(stagebeta2mean))
            V2m = V1m*dehallerm

            deltawhirlh = ((1.005*(10**3)*(stagedeltaTo))/(lambcomp*Uhcomp))
            Cw2h = deltawhirlh - Cw1h
            stagebeta1hub = np.degrees(np.arctan(Uhcomp/Castage))
            stagebeta2hub = np.degrees(np.arctan((Uhcomp-Cw2h)/Castage))
            stagealpha1hub = 0
            stagealpha2hub = np.degrees(np.arctan((Cw2h)/Castage))

            V1h = Castage/(np.cos(np.radians(stagebeta1hub)))
            dehallerh = np.cos(np.radians(stagebeta1hub))/np.cos(np.radians(stagebeta2hub))
            V2h = V1h*dehallerh
        else: """
        stagebeta2tip,stagebeta3tip,stagealpha2tip,stagealpha3tip,Cw2t,Cw3t,V2t,V3t,bladelt,flowt = returnstage(deltaTosact,Utstage,Ca,DOR)

        stagebeta2mean,stagebeta3mean,stagealpha2mean,stagealpha3mean,Cw2m,Cw3m,V2m,V3m,bladelm,flowm = returnstage(deltaTosact,Umstage,Ca,DOR)

        stagebeta2hub,stagebeta3hub,stagealpha2hub,stagealpha3hub,Cw2h,Cw3h,V2h,V3h,bladelh,flowh = returnstage(deltaTosact,Uhstage,Ca,DOR)

        #pr = (1+((polyc*(((Umstage*Castage)/(cpa*1000))*(np.tan(np.radians(stagebeta1mean))-np.tan(np.radians(stagebeta2mean)))))/(stagevals[i-1,0,6])))**(gammac/(gammac-1))
        stagepo3 = stagevals[i-1,0,15]*pr

        stagevals[i,:,:] = np.array([[stagealpha2hub,stagealpha3hub,stagebeta2hub,stagebeta3hub,rhstage,deltaTosact,stageTo3,Uhstage,Ca,Cw2h,Cw3h,V2h,V3h,pr,stagepo3,bladelh,flowh],[stagealpha2mean,stagealpha3mean,stagebeta2mean,stagebeta3mean,rmstage,deltaTosact,stageTo3,Umstage,Ca,Cw2m,Cw3m,V2m,V3m,pr,stagepo3,bladelm,flowm],[stagealpha2tip,stagealpha3tip,stagebeta2tip,stagebeta3tip,rtstage,deltaTosact,stageTo3,Utstage,Ca,Cw2t,Cw3t,V2t,V3t,pr,stagepo3,bladelt,flowt]])

        i += 1

    return stagevals

In [995]:
def valtable(location,vals):
    if location == 0:
        print("Hub Values:")
    elif location == 1:
        print("Mean Values:")
    elif location == 2:
        print("Tip Values:")
    
    num = 0
    stages = ["Inlet"]

    while num < len(vals[:,location,4])-1:
        stages.append("Stage "+str(num+1))
        num += 1

    pd.set_option("display.precision", 4)
    tabletry = pd.DataFrame({"Alpha 2": vals[:,location,0],"Alpha 3": vals[:,location,1],"Beta 2": vals[:,location,2],"Beta 3": vals[:,location,3],"Radius": vals[:,location,4],"Delta To": vals[:,location,5],"To2": vals[:,location,6],"U": vals[:,location,7],"Ca": vals[:,location,8],"Cw1": vals[:,location,9],"Cw2": vals[:,location,10],"V1": vals[:,location,11],"V2": vals[:,location,12],"PR": vals[:,location,13],"po2": vals[:,location,14],"Blade Loading": vals[:,location,15],"Flow": vals[:,location,16]},index=stages)

    return tabletry

In [996]:
#valtable(0,stagevals)

In [997]:
#valtable(1,stagevals)

In [998]:
#valtable(2,stagevals)

In [999]:
check = False

In [1000]:
end = 0
Ca = 150
while not check:
    HTratio = 0.3
    while HTratio < .8:
        N = 220
        while N < 250:
            deltaToturb,r4t,r4h,r4m = initialvals4(deltaTocomp,Ca,To4,po4,HTratio)
            r5t,r5h,r5m = initialvals5(deltaToturb,Ca,To4,po4,r4h)
            stageest = estimatevals(r5h,N,Ca,3.3,deltaToturb)
            stageactuala = np.ceil(stageest).astype('int32')
            # Values per stage: alpha2, alpha3, beta2, beta3, r, deltaT, T3, U, Cw2, Cw3, V2, V3, pr, po3, bladeloading, flowcoeff
            stagevalsa = np.zeros((stageactuala+1,3,17))
            stagevalsa[0,:,:] = np.array([[0,0,0,0,0,0,To4,U4h,Ca,0,0,0,0,0,po4,0,0],[0,0,0,0,0,0,To4,U4m,Ca,0,0,0,0,0,po4,0,0],[0,0,0,0,0,0,To4,U4t,Ca,0,0,0,0,0,po4,0,0]])
            deltaTosact = deltaToturb/stageactuala
            stagevalsa = iterate(stagevalsa,stageactuala,deltaTosact,r4t,r4m,r4h,N,Ca)
            
            if stagevalsa[1,0,15] < 3.3 and stagevalsa[stageactuala,2,16] > 0.78 and stagevalsa[1,0,16] > 0.78 and stagevalsa[stageactuala,2,7] < 290:
                print("Stagecount:"+str(stageactuala))
                print(len(stagevalsa[:,0,4]))
                print(stagevalsa)
                check = True
            N += 1
        HTratio += .1

    Ca += 0.1
    if end > 10000:
        check = True
        print("BAH")
    end += 1

print(HTratio)
print(N)
print(Ca)
print(end)

Stagecount:57
58
[[[  0.           0.           0.         ...  15.           0.
     0.        ]
  [  0.           0.           0.         ...  15.           0.
     0.        ]
  [  0.           0.           0.         ...  15.           0.
     0.        ]]

 [[ 16.56224689  10.21011284   0.23967451 ...   0.           3.25706624
     3.41049603]
  [ 13.35017245  -1.01516271 -21.81482523 ...   0.           0.6888321
     1.56841483]
  [ 14.98579779  -7.1306919  -35.53734044 ...   0.           0.29040466
     1.01837105]]

 [[ 16.56224689  10.21011284   0.23967451 ...   3.28789187   3.25706624
     3.41049603]
  [ 13.35370532  -1.06395988 -21.92474667 ...   3.28789187   0.68390471
     1.56279514]
  [ 15.0171089   -7.20110948 -35.68879992 ...   3.28789187   0.28771135
     1.0136377 ]]

 ...

 [[ 16.56224689  10.21011284   0.23967451 ...   3.28373716   3.25706624
     3.41049603]
  [ 13.72426464  -3.45274102 -27.34667308 ...   3.28373716   0.48302964
     1.31338225]
  [ 16.84890441 -

In [1001]:
valtable(0,stagevalsa)

Hub Values:


,Alpha 2,Alpha 3,Beta 2,Beta 3,Radius,Delta To,To2,U,Ca,Cw1,Cw2,V1,V2,PR,po2,Blade Loading,Flow
Inlet,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,2275.0000,212.5816,234.1,0.0000,0.0000,0.0000,0.0000,0.0000,15.0000,0.0000,0.0000
Stage 1,50.8142,33.5817,-10.2189,64.2255,0.2105,126.9919,2401.9919,329.3811,234.1,287.1802,155.4278,237.8733,538.3702,1.1916,0.0000,2.6875,0.7107
Stage 2,50.8142,33.5817,-10.2189,64.2255,0.2105,126.9919,2528.9838,329.3811,234.1,287.1802,155.4278,237.8733,538.3702,1.1808,3.1735,2.6875,0.7107
Stage 3,53.8223,27.6202,-2.2659,62.6128,0.2105,126.9919,2655.9757,329.3811,234.1,320.1183,122.4896,234.2832,508.9106,1.1712,3.1477,2.6875,0.7107


In [1002]:
valtable(1,stagevalsa)


Mean Values:


,Alpha 2,Alpha 3,Beta 2,Beta 3,Radius,Delta To,To2,U,Ca,Cw1,Cw2,V1,V2,PR,po2,Blade Loading,Flow
Inlet,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,2275.0000,249.9270,234.1,0.0000,0.0000,0.0000,0.0000,0.0000,15.0000,0.0000,0.0000
Stage 1,48.7792,26.0452,-26.1237,64.7528,0.2442,126.9919,2401.9919,382.0190,234.1,267.2146,114.4070,260.7352,548.8548,1.1916,0.0000,1.9979,0.6128
Stage 2,48.4296,24.4734,-28.9574,64.9127,0.2515,126.9919,2528.9838,393.4841,234.1,263.9479,106.5542,267.5490,552.1242,1.1808,3.1735,1.8832,0.5949
Stage 3,52.1716,14.0354,-23.8426,63.2016,0.2588,126.9919,2655.9757,404.9493,234.1,301.4909,58.5213,255.9423,519.2378,1.1712,3.1477,1.7781,0.5781


In [1003]:
valtable(2,stagevalsa)

Tip Values:


,Alpha 2,Alpha 3,Beta 2,Beta 3,Radius,Delta To,To2,U,Ca,Cw1,Cw2,V1,V2,PR,po2,Blade Loading,Flow
Inlet,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,2275.0000,287.2724,234.1,0.0000,0.0000,0.0000,0.0000,0.0000,15.0000,0.0000,0.0000
Stage 1,47.4059,19.0361,-37.5602,65.5732,0.2778,126.9919,2401.9919,434.6568,234.1,254.6346,80.7719,295.3147,566.1003,1.1916,0.0000,1.5433,0.5386
Stage 2,46.9744,16.1480,-41.4528,65.9827,0.2925,126.9919,2528.9838,457.5872,234.1,250.8168,67.7820,312.3408,575.1656,1.1808,3.1735,1.3925,0.5116
Stage 3,51.6464,1.8453,-38.2674,64.3751,0.3071,126.9919,2655.9757,480.5175,234.1,295.8529,7.5423,298.1675,541.2996,1.1712,3.1477,1.2628,0.4872
